In [6]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam,SGD,RMSprop,Adadelta
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time

In [7]:
main_path_energy ='../data_base/paths_energy/'
main_path_total_energy = '../data_base/paths_totals/'

energy_path11_pd = pd.read_csv(main_path_energy + 'feat_pathv11.csv')
energy_path12_pd = pd.read_csv(main_path_energy + 'feat_pathv12-2.csv')
energy_path13_pd = pd.read_csv(main_path_energy + 'feat_pathv13.csv')

energy_path_list = [energy_path11_pd,energy_path12_pd,energy_path13_pd]
complete_energy_pd = pd.concat(energy_path_list)

total_path11_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv11.csv')
total_path12_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv12-2.csv')
total_path13_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv13.csv')

total_path_list = [total_path11_pd,total_path12_pd,total_path13_pd]
complete_total_pd = pd.concat(total_path_list)

In [8]:
eps = 5
complete_energy_pd = complete_energy_pd[(complete_energy_pd['sim_point_dist']-complete_energy_pd['teo_point_dist']) < eps]
print(f'new shape of complete_energy_pd = {complete_energy_pd.shape}')

new shape of complete_energy_pd = (2577, 16)


In [9]:
X = complete_energy_pd.drop(columns=["missing_points","path_num","sim_drone_time","Energy","sim_point_dist","sim_Xdist","sim_Ydist","avg_error_target_dist","avg_energy"],axis=1)
y = complete_energy_pd["Energy"].to_numpy()

X = X.to_numpy()

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=35)

scaler2 = StandardScaler()

x_train = scaler2.fit_transform(x_train)
x_test = scaler2.transform(x_test)

print(f'Train size/Test size {len(x_train),len(x_test)}')

Train size/Test size (2061, 516)


#### Pruebas con Optimizer: Adam,variando lr

In [10]:
start_time = time.time()
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=7,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

lr = [0.1,0.01,0.001,0.0001,0.00001]
results_Adam = np.empty((4,5),dtype=object)

for idx,i in enumerate(lr):
    NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=Adam(learning_rate=i), metrics=["mean_squared_logarithmic_error"])
    model = NN_model.fit(x_train, y_train, epochs=250, batch_size=32, validation_split = 0.2)
    
    y_predict = NN_model.predict(x_test)
    r2 = r2_score(y_test,y_predict)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MSE = mean_squared_error(y_test,y_predict)
    
    results_Adam[0,idx] = r2
    results_Adam[1,idx] = RMSE
    results_Adam[2,idx] = MSE
    results_Adam[3,idx] = model

print(results_Adam)
print('Execution time = ' + str(time.time() - start_time))

Train on 1648 samples, validate on 413 samples
Epoch 1/250
1648/1648 [==============================] - 0s 278us/sample - loss: 203.5789 - mean_squared_logarithmic_error: 203.5790 - val_loss: 203.4677 - val_mean_squared_logarithmic_error: 203.4677
Epoch 2/250
1648/1648 [==============================] - 0s 92us/sample - loss: 203.5789 - mean_squared_logarithmic_error: 203.5790 - val_loss: 203.4677 - val_mean_squared_logarithmic_error: 203.4677
Epoch 3/250
1648/1648 [==============================] - 0s 101us/sample - loss: 203.5789 - mean_squared_logarithmic_error: 203.5790 - val_loss: 203.4677 - val_mean_squared_logarithmic_error: 203.4677
Epoch 4/250
1648/1648 [==============================] - 0s 101us/sample - loss: 203.5789 - mean_squared_logarithmic_error: 203.5789 - val_loss: 203.4677 - val_mean_squared_logarithmic_error: 203.4677
Epoch 5/250
1648/1648 [==============================] - 0s 106us/sample - loss: 203.5789 - mean_squared_logarithmic_error: 203.5789 - val_loss: 203.4

In [11]:
print(results_Adam)

[[-1.6558585816509628 -1.6558585816509628 -1.6558585816509628
  -1.6558585816509628 -1.6558585816509628]
 [2333157.882182092 2333157.882182092 2333157.882182092 2333157.882182092
  2333157.882182092]
 [5443625703188.425 5443625703188.425 5443625703188.425 5443625703188.425
  5443625703188.425]


#### SGD with momentum, variando el lr y dejando el momemtum en 0.9

In [12]:
start_time = time.time()
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=7,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

lr = [0.1,0.01,0.001,0.0001,0.00001]
results_SGD = np.empty((4,5),dtype=object)

for idx,i in enumerate(lr):
    NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=SGD(learning_rate=i,momentum=0.9), metrics=["mean_squared_logarithmic_error"])
    model = NN_model.fit(x_train, y_train, epochs=250, batch_size=32, validation_split = 0.2,verbose=0)
    
    y_predict = NN_model.predict(x_test)
    r2 = r2_score(y_test,y_predict)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MSE = mean_squared_error(y_test,y_predict)
    
    results_SGD[0,idx] = r2
    results_SGD[1,idx] = RMSE
    results_SGD[2,idx] = MSE
    results_SGD[3,idx] = model
print('Execution time = ' + str(time.time() - start_time))

Execution time = 151.99165892601013


In [13]:
print(results_SGD)

[[0.7320034437984028 0.7339845814047361 0.7340218411029843
  0.73400607915679 0.7340050533443576]
 [741150.2536880452 738405.7327506045 738354.0181951091 738375.8954122916
  738377.3191937837]
 [549303698541.8538 545243026158.95715 545166656184.8635
  545198962925.9034 545201065499.79865]


#### RMSprop variando el learning rate, momentum 0.9

In [14]:
start_time = time.time()
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=7,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

lr = [0.1,0.01,0.001,0.0001,0.00001]
results_RMSprop = np.empty((4,5),dtype=object)

for idx,i in enumerate(lr):
    NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=RMSprop(learning_rate=i,momentum=0.9), metrics=["mean_squared_logarithmic_error"])
    model = NN_model.fit(x_train, y_train, epochs=250, batch_size=32, validation_split = 0.2,verbose=0)
    
    y_predict = NN_model.predict(x_test)
    r2 = r2_score(y_test,y_predict)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MSE = mean_squared_error(y_test,y_predict)
    
    results_RMSprop[0,idx] = r2
    results_RMSprop[1,idx] = RMSE
    results_RMSprop[2,idx] = MSE
    results_RMSprop[3,idx] = model

print('Execution time = ' + str(time.time() - start_time))

Execution time = 171.50318694114685


In [15]:
print(results_RMSprop)

[[-10025.524023711543 -10025.524023711543 -10025.524023711543
  -10025.524023711543 -10025.524023711543]
 [143356323.40543306 143356323.40543306 143356323.40543306
  143356323.40543306 143356323.40543306]
 [2.055103546032311e+16 2.055103546032311e+16 2.055103546032311e+16
  2.055103546032311e+16 2.055103546032311e+16]


#### Adadelta, variando el lr

In [16]:
start_time = time.time()
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=7,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

lr = [0.1,0.01,0.001,0.0001,0.00001]
results_Adadel = np.empty((4,5),dtype=object)

for idx,i in enumerate(lr):
    NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=Adadelta(learning_rate=i), metrics=["mean_squared_logarithmic_error"])
    model = NN_model.fit(x_train, y_train, epochs=250, batch_size=32, validation_split = 0.2,verbose=0)
    
    y_predict = NN_model.predict(x_test)
    r2 = r2_score(y_test,y_predict)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MSE = mean_squared_error(y_test,y_predict)
    
    results_Adadel[0,idx] = r2
    results_Adadel[1,idx] = RMSE
    results_Adadel[2,idx] = MSE
    results_Adadel[3,idx] = model
print('Execution time = ' + str(time.time() - start_time))

Execution time = 200.72292828559875


In [17]:
print(results_Adadel)

[[0.7365143668992729 0.7366177619973031 0.7366203682738974
  0.7366191034342224 0.7366195439867853]
 [734886.2553501532 734742.0518669939 734738.4165691765 734740.1808004845
  734739.5663067808]
 [540057808302.5706 539845882781.72034 539840540782.5807
  539843133282.7286 539842230296.6764]


In [18]:
results_list = [results_Adam,results_Adadel,results_RMSprop,results_SGD]
results = [i[0,:] for i in results_list]
print(np.max(results))

0.7366203682738974


In [19]:
results

[array([-1.6558585816509628, -1.6558585816509628, -1.6558585816509628,
        -1.6558585816509628, -1.6558585816509628], dtype=object),
 array([0.7365143668992729, 0.7366177619973031, 0.7366203682738974,
        0.7366191034342224, 0.7366195439867853], dtype=object),
 array([-10025.524023711543, -10025.524023711543, -10025.524023711543,
        -10025.524023711543, -10025.524023711543], dtype=object),
 array([0.7320034437984028, 0.7339845814047361, 0.7340218411029843,
        0.73400607915679, 0.7340050533443576], dtype=object)]